In [12]:
#!pip install transformers torch datasets evaluate accelerate scikit-learn

In [33]:
# Loading Datasets
from datasets import load_dataset

imdb = load_dataset("imdb")


#Sampling data (to make it faster)
from random import sample
from datasets import DatasetDict, Dataset

sample_indeces = sample(range(1,25000),100)
imdb["train"] = {"text" : [imdb["train"][index]["text"] for index in sample_indeces],
                    "label" : [imdb["train"][index]["label"] for index in sample_indeces]}
imdb["test"] ={"text" : [imdb["test"][index]["text"] for index in sample_indeces],
                    "label" : [imdb["test"][index]["label"] for index in sample_indeces]}
imdb = DatasetDict({"train":Dataset.from_dict(imdb["train"]),
                    "test" : Dataset.from_dict(imdb["test"])})

In [34]:
#Tokenizing
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)
tokenized_imdb = imdb.map(preprocess_function, batched=True)

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [35]:
#Eval and stuff

from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [36]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny", num_labels=2, id2label=id2label, label2id=label2id
)

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
training_args = TrainingArguments(
    output_dir="../serverless_function/model",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/chemmer/.pyenv/versions/3.11.8/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.678053,0.580000
2,No log,0.676832,0.580000


TrainOutput(global_step=50, training_loss=0.6768798828125, metrics={'train_runtime': 11.6055, 'train_samples_per_second': 17.233, 'train_steps_per_second': 4.308, 'total_flos': 219208642800.0, 'train_loss': 0.6768798828125, 'epoch': 2.0})

In [38]:
trainer.save_model("../serverless_function/model/")